In [1]:
from my_models import ResNet101, xception

In [1]:
import os
import easydict
from PIL import Image as im
from PIL import ImageOps
import cv2
from tqdm import tqdm
import json
import sys
import shutil
from datetime import datetime
import pandas as pd
import random

from pycocotools.coco import COCO

op = os.path.join

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
import torchvision.models as models

# from efficientnet_pytorch import EfficientNet

ImportError: cannot import name 'COCO' from 'pycocotools' (/home/dmsai2/.conda/envs/openmmlab/lib/python3.8/site-packages/pycocotools/__init__.py)

In [5]:
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score
from tensorboardX import SummaryWriter

In [8]:
DATA_DIR = "/home/dmsai2/mmdetection/data/"

TRAIN_DATASET_DIR = op(DATA_DIR, "classification")
TRAIN_IMAGE_DIR = op(TRAIN_DATASET_DIR, "train")
TRAIN_JSON_DIR = op(TRAIN_DATASET_DIR, "annotations")

TEST_DATASET_DIR = op(DATA_DIR, "classification")
TEST_IMAGE_DIR = op(TEST_DATASET_DIR, "test")
TEST_JSON_DIR = op(TEST_DATASET_DIR, "annotations")

current_datetime = datetime.now()
formatted_datetime = current_datetime.strftime('%Y-%m-%d_%H_%M')
print(formatted_datetime)

os.makedirs(op("/home/dmsai2/mmdetection/work_dir/classification/logs", formatted_datetime), exist_ok=True)
os.makedirs(op("/home/dmsai2/mmdetection/work_dir/classification/weights", formatted_datetime), exist_ok=True)

2024-06-19_06_42


In [9]:
train_file_list = list(map(lambda x : x.split(".")[0], os.listdir(TRAIN_IMAGE_DIR)))
train_image_list = os.listdir(TRAIN_IMAGE_DIR)
train_json_list = os.listdir(TRAIN_JSON_DIR)
print("number of train file:", len(train_file_list))

test_file_list = list(map(lambda x : x.split(".")[0], os.listdir(TEST_IMAGE_DIR)))
test_image_list = os.listdir(TEST_IMAGE_DIR)
test_json_list = os.listdir(TEST_JSON_DIR)
print("number of train file:", len(test_file_list))

number of train file: 1991
number of train file: 250


In [10]:
# train_df = pd.read_csv("/hoeunlee228/Dataset/train_df.csv")
# # print(train_df.head())
# train_df['is_decayed'].value_counts()

# not_decayed_rows = train_df[train_df['is_decayed'] == False]
# decayed_rows = train_df[train_df['is_decayed'] == True]
# print("total not decayed rows:", len(not_decayed_rows), "total decayed rows", len(decayed_rows))

# num_samples_not_decayed = 28136 * 3
# num_samples_decayed = 28136

# print("args num sampled not decayed:", num_samples_not_decayed, "args num sampled decayed:", num_samples_decayed)

# random_samples_not_decayed = random.sample(range(len(not_decayed_rows)), num_samples_not_decayed)
# not_decayed_sampled_df = not_decayed_rows.iloc[random_samples_not_decayed]
# print("num of not decayed sampled list", len(not_decayed_sampled_df))

# random_samples_decayed = random.sample(range(len(decayed_rows)), num_samples_decayed)
# decayed_sampled_df = decayed_rows.iloc[random_samples_decayed]
# print("num of decayed sampled list", len(decayed_sampled_df))

# train_file_list_not_decayed_sampled = list(map(lambda x : f"{x[0]}_{x[1]}", not_decayed_sampled_df[['file', 'teeth_idx']].values.tolist()))
# train_file_list_decayed_sampled = list(map(lambda x : f"{x[0]}_{x[1]}", decayed_sampled_df[['file', 'teeth_idx']].values.tolist()))
# print("num of sampled file list (not decayed, decayed):", len(train_file_list_not_decayed_sampled), len(train_file_list_decayed_sampled))

# sampled_train_list = train_file_list_not_decayed_sampled + train_file_list_decayed_sampled
# print("total num of sampled train list:", len(sampled_train_list))

In [11]:
# class ToothDataset(Dataset):
#     def __init__(self, data_dir, file_list, transform=None, aug_transform=None):
#         self.data_dir = data_dir
#         self.file_list = file_list
#         self.transform = transform
#         self.aug_transform = aug_transform

#     def __len__(self):
#         return len(self.file_list)
    
#     def _load_image(self, image_path):
#         assert os.path.exists(op(self.data_dir, "image", image_path))
#         # return cv2.cvtColor(cv2.imread(op(self.data_dir, "image", image_path)), cv2.COLOR_BGR2RGB)
#         return im.open(op(self.data_dir, "image", image_path)).convert("RGB")
    
#     def __getitem__(self, index):
#         image_path = self.file_list[index] + ".png"
#         json_path = self.file_list[index] + ".json"

#         image = self._load_image(image_path)

#         with open(op(self.data_dir, "json", json_path), 'r') as json_file:
#             data = json.load(json_file)

#         decayed = data["tooth"][0]["decayed"]
#         target = 1 if decayed else 0

#         if self.transform:
#             image = self.transform(image)

#         if self.aug_transform:
#             image = self.aug_transform(image)

#         return image, target

In [12]:
class ToothCOCODataset(Dataset):
    def __init__(self, data_dir, ann_file, transform=None, aug_transform=None, valid=False):
        self.data_dir = data_dir
        self.coco = COCO(ann_file)
        self.image_ids = self.coco.getImgIds()
        self.transform = transform
        self.aug_transform = aug_transform
        self.valid = valid

    def __len__(self):
        return len(self.image_ids)
    
    def _load_image(self, image_id):
        image_info = self.coco.loadImgs(image_id)[0]
        image_path = op(self.data_dir, image_info['file_name'])
        assert os.path.exists(image_path), f"Image path {image_path} does not exist."
        
        if self.valid:
            return (im.open(image_path).convert("RGB"), image_info['file_name'])
        else:
            return im.open(image_path).convert("RGB")
    
    def _load_target(self, image_id):
        ann_ids = self.coco.getAnnIds(imgIds=image_id)
        anns = self.coco.loadAnns(ann_ids)
        # Assuming 'decayed' is the attribute that indicates decay
        # decayed = any(ann.get('category_id', False) for ann in anns)
        decayed = any(ann.get('category_id', False) for ann in anns)
        target = 1 if decayed else 0
        return target

    def __getitem__(self, index):
        image_id = self.image_ids[index]

        if self.valid:
            image, label = self._load_image(image_id)
        else:
            image = self._load_image(image_id)

        target = self._load_target(image_id)

        if self.transform:
            image = self.transform(image)

        if self.aug_transform:
            image = self.aug_transform(image)

        if self.valid:
            return (image, label), target
        else:
            return image, target

In [13]:
# 1. Zero padding to make the image square
def make_square(img):
    width, height = img.size
    max_side = max(width, height)
    left = (max_side - width) // 2
    top = (max_side - height) // 2
    right = (max_side - width) - left
    bottom = (max_side - height) - top
    padding = (left, top, right, bottom)
    return ImageOps.expand(img, padding)

In [14]:
mean = (0.57933619, 0.42688786, 0.33401168)
std = (0.35580848, 0.27125023, 0.22251333)

test_aug_transform = transforms.Compose([
    # 1. 이미지를 정사각형으로 만들기
    transforms.Lambda(lambda img: ImageOps.exif_transpose(img)),  # Exif 정보 처리
    transforms.Lambda(make_square),

    # 2. Resize
    transforms.Resize(size=(300, 300)),
    
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

valid_aug_transform = transforms.Compose([
    # 1. 이미지를 정사각형으로 만들기
    transforms.Lambda(lambda img: ImageOps.exif_transpose(img)),  # Exif 정보 처리
    transforms.Lambda(make_square),

    # 2. Resize
    transforms.Resize(size=(300, 300)),

    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [15]:
args = easydict.EasyDict({    
    "gpu": 0,
    # 데이터 위치
    "root": "/home/dmsai2/mmdetection/data/classification/",
    # pth
    "save_fn": "/home/dmsai2/mmdetection/image-classification-fastapi/xception_epoch26.pth"
})

assert os.path.isfile(args.save_fn), 'wrong path'

In [16]:
model = xception(num_out_classes=2, dropout=0.5)
print("=> creating model '{}'".format('xception'))
model = model.cuda(args.gpu)

=> creating model 'xception'


In [18]:
model.load_state_dict(torch.load(args.save_fn)['state_dict'])
print("=> model weight '{}' is loaded".format(args.save_fn))

model = model.eval()

=> model weight '/home/dmsai2/mmdetection/image-classification-fastapi/xception_epoch26.pth' is loaded


In [20]:
# predict label
m = nn.Softmax()

with torch.no_grad():
    image_path = "/home/dmsai2/mmdetection/image-classification-fastapi/front_100_11.png"
    image = im.open(image_path).convert("RGB")
    image = valid_aug_transform(image)
    image = torch.unsqueeze(image, 0)
    image = image.cuda(args.gpu, non_blocking=True)
    
    output = model(image)
    output = m(output)[0]
    
    print(f"tooth: {output[0]}, cavity: {output[1]}")

tooth: 0.8735164999961853, cavity: 0.12648352980613708


/home/dmsai2/.conda/envs/openmmlab/lib/python3.8/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [ ]:
# collect all images
print(len(test_file_list))

# prepare pred dict
test_predict_dict = {}
image_inf_list = []

for f in os.listdir("/hoeunlee228/Dataset/test_data/image/"):
    test_predict_dict[f] = []

# predict label
m = nn.Softmax()

with torch.no_grad():
    for tf in tqdm(test_file_list):
        image_path = op(TEST_DATASET_DIR, "image", tf + ".png")
        image = im.open(image_path).convert('RGB')
        image = valid_aug_transform(image)
        image = torch.unsqueeze(image, dim=0)
        image = image.cuda(args.gpu, non_blocking=True)

        original_f = tf.split("_")[0] + "_" + tf.split("_")[1] + ".png"
        file_name = tf.split("_")[0] + "_" + tf.split("_")[1]
        teeth_idx = int(tf.split("_")[2])

        output = model(image)
        output = m(output)[0]  # apply softmax

        # 0 = not decayed
        # write to submission file
        if output[0] > output[1]:
            test_predict_dict[original_f].append(False)
            image_inf_list.append([file_name, teeth_idx, False, (output[0].cpu().item(), output[1].cpu().item())])

        # 1 = decayed
        else:
            test_predict_dict[original_f].append(True)
            image_inf_list.append([file_name, teeth_idx, True, (output[0].cpu().item(), output[1].cpu().item())])

print("predicted finished:", len(test_predict_dict.keys()))
print("len of preds:", len(image_inf_list))

In [24]:
model = EfficientNet.from_name('efficientnet-b3')
print("=> creating model '{}'".format('efficientnet-b3'))
model = model.cuda(args.gpu)

# assert os.path.isfile(args.save_fn), 'wrong path'

model.load_state_dict(torch.load(args.save_fn)['state_dict'])
print("=> model weight '{}' is loaded".format(args.save_fn))

model = model.eval()

=> creating model 'efficientnet-b3'
=> model weight '/hoeunlee228/weights/2023-12-16_02_49/efficientnetb3_epoch5.pth' is loaded


In [25]:
test_df = pd.read_csv("/hoeunlee228/test_df.csv", index_col=0)
test_df.head()

,file,teeth_idx,num_seg,is_decayed,bound_x,bound_size_x,bound_y,bound_size_y,data_type,image_type,is_side
0,front_100,11,974,False,"(561, 925)",364,"(0, 396)",396,test,front,False
1,front_100,12,790,False,"(312, 573)",261,"(35, 368)",333,test,front,True
2,front_100,13,713,False,"(101, 329)",228,"(69, 411)",342,test,front,True
3,front_100,14,459,False,"(0, 167)",167,"(183, 413)",230,test,front,True
4,front_100,21,929,False,"(909, 1281)",372,"(48, 408)",360,test,front,False


In [44]:
# collect all images
print(len(test_file_list))

# prepare pred dict
test_predict_dict = {}
image_inf_list = []

for f in os.listdir("/hoeunlee228/Dataset/test_data/image/"):
    test_predict_dict[f] = []

# predict label
m = nn.Softmax()

with torch.no_grad():
    for tf in tqdm(test_file_list):
        image_path = op(TEST_DATASET_DIR, "image", tf + ".png")
        image = im.open(image_path).convert('RGB')
        image = valid_aug_transform(image)
        image = torch.unsqueeze(image, dim=0)
        image = image.cuda(args.gpu, non_blocking=True)

        original_f = tf.split("_")[0] + "_" + tf.split("_")[1] + ".png"
        file_name = tf.split("_")[0] + "_" + tf.split("_")[1]
        teeth_idx = int(tf.split("_")[2])

        output = model(image)
        output = m(output)[0]  # apply softmax

        # 0 = not decayed
        # write to submission file
        if output[0] > output[1]:
            test_predict_dict[original_f].append(False)
            image_inf_list.append([file_name, teeth_idx, False, (output[0].cpu().item(), output[1].cpu().item())])

        # 1 = decayed
        else:
            test_predict_dict[original_f].append(True)
            image_inf_list.append([file_name, teeth_idx, True, (output[0].cpu().item(), output[1].cpu().item())])

print("predicted finished:", len(test_predict_dict.keys()))
print("len of preds:", len(image_inf_list))

37489


  0%|          | 0/37489 [00:00<?, ?it/s]/tmp/ipykernel_55838/644755056.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = m(output)[0]  # apply softmax
100%|██████████| 37489/37489 [16:01<00:00, 39.00it/s]

predicted finished: 3000
len of preds: 37489


In [45]:
correct_count_image = 0
inf_list_image = []

for row in tqdm(image_inf_list):
    # name, pred, answer, same
    file_name = row[0]
    teeth_idx = row[1]
    f_pred = row[2]
    # print(file_name, teeth_idx)
    f_answer = test_df[(test_df['file'] == file_name) & (test_df['teeth_idx'] == teeth_idx)]['is_decayed'].values[0]
    correct = (f_pred == f_answer)
    # print(file_name, f_pred, f_answer, correct)
    inf_list_image.append([file_name, f_pred, f_answer, correct])

    if correct:
        correct_count_image += 1

print(f"{correct_count_image}/{len(inf_list_image)}")

100%|██████████| 37489/37489 [01:49<00:00, 342.18it/s]

35443/37489


In [54]:
# Extract labels and predictions
labels_image = [item[2] for item in inf_list_image]
predictions_image = [item[1] for item in inf_list_image]

In [55]:
print(labels_image.count(0) + labels_image.count(1))
print(labels_image.count(0), labels_image.count(1))

print(predictions_image.count(0) + predictions_image.count(1))
print(predictions_image.count(0), predictions_image.count(1))

# Convert boolean values to integers (True -> 1, False -> 0)
labels_image = [int(label) for label in labels_image]
predictions_image = [int(prediction) for prediction in predictions_image]

37489
36489 1000
37489
35073 2416


In [56]:
# Calculate metrics
accuracy_image = accuracy_score(labels_image, predictions_image)
precision_image = precision_score(labels_image, predictions_image)
recall_image = recall_score(labels_image, predictions_image)
auc_image = roc_auc_score(labels_image, predictions_image)
f1_image = f1_score(labels_image, predictions_image)

print("Accuracy:", accuracy_image)
print("Precision:", precision_image)
print("Recall:", recall_image)
print("AUC:", auc_image)
print("F1 Score:", f1_image)

Accuracy: 0.945423991037371
Precision: 0.28352649006622516
Recall: 0.685
AUC: 0.8187805228973115
F1 Score: 0.4010538641686183


In [57]:
# Calculate confusion matrix
conf_matrix_image = confusion_matrix(labels_image, predictions_image)
print(conf_matrix_image)

# Extract TP, FP, TN, FN
tp = conf_matrix_image[1, 1]
fp = conf_matrix_image[0, 1]
tn = conf_matrix_image[0, 0]
fn = conf_matrix_image[1, 0]

print("True Positive (TP):", tp)
print("False Positive (FP):", fp)
print("True Negative (TN):", tn)
print("False Negative (FN):", fn)

[[34758  1731]
 [  315   685]]
True Positive (TP): 685
False Positive (FP): 1731
True Negative (TN): 34758
False Negative (FN): 315


In [58]:
test_image_df = pd.read_csv("/hoeunlee228/test_image_df.csv")
print("total num of test data:", len(test_image_df))
test_image_df['decayed'].value_counts()

total num of test data: 3000


False    2000
True     1000
Name: decayed, dtype: int64

In [59]:
correct_count = 0
inf_list = []

for index, row in tqdm(test_image_df.iterrows()):
    # name, pred, answer, same
    file_name = row['name']
    f_answer = row['decayed']
    f_pred = any(test_predict_dict[row['name'] + ".png"])
    correct = (f_pred == f_answer)
    # print(file_name, f_pred, f_answer, correct)
    inf_list.append([file_name, f_pred, f_answer, correct])

    if correct:
        correct_count += 1

print(f"{correct_count}/{len(test_image_df)}")

3000it [00:00, 22070.06it/s]

2218/3000


In [60]:
# Extract labels and predictions
labels = [item[2] for item in inf_list]
predictions = [item[1] for item in inf_list]

# Convert boolean values to integers (True -> 1, False -> 0)
labels = [int(label) for label in labels]
predictions = [int(prediction) for prediction in predictions]

# Calculate metrics
accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions)
recall = recall_score(labels, predictions)
auc = roc_auc_score(labels, predictions)
f1 = f1_score(labels, predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("AUC:", auc)
print("F1 Score:", f1)

Accuracy: 0.7393333333333333
Precision: 0.5774147727272727
Recall: 0.813
AUC: 0.7577499999999999
F1 Score: 0.6752491694352158


In [61]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(labels, predictions)
print(conf_matrix)

# Extract TP, FP, TN, FN
tp = conf_matrix[1, 1]
fp = conf_matrix[0, 1]
tn = conf_matrix[0, 0]
fn = conf_matrix[1, 0]

print("True Positive (TP):", tp)
print("False Positive (FP):", fp)
print("True Negative (TN):", tn)
print("False Negative (FN):", fn)

[[1405  595]
 [ 187  813]]
True Positive (TP): 813
False Positive (FP): 595
True Negative (TN): 1405
False Negative (FN): 187


: 